# Voila/papermill notebook 

1. Choose a notebook
2. Set parameters (parameters extracted by papermill; GUI generated by ipywidgets)
3. Run the notebook with those parameters

In [ ]:
import os
import urllib
import ipywidgets as widgets
from papermill import inspect_notebook
from papermill.parameterize import parameterize_notebook
from papermill.iorw import load_notebook_node, list_notebook_files
from IPython.display import display, clear_output, Markdown, HTML

In [ ]:
# Choose notebooks from the current directory
notebooks = [os.path.basename(f) for f in list_notebook_files(os.getcwd())]
notebooks.remove('notebook-runner.ipynb')
notebooks.sort()

In [ ]:
# Run the target notebook.  Each cell is executed by the current
# kernel and captured by an Output widget.  Markdown cells are
# rendering using IPython.display.  Other cell types are ignored.
def run_target(target, params):
    target_nb = load_notebook_node(target)
    target_nb = parameterize_notebook(target_nb, params)
    target_output.clear_output()
    
    for cell in target_nb.cells:
        cell_output = widgets.Output() #layout={'border': '1px solid black'})
        if cell.cell_type == 'code':
            with cell_output:
                get_ipython().ex(cell.source)
        elif cell.cell_type == 'markdown':
            with cell_output:
                display(Markdown(cell.source))
        with target_output:
            display(cell_output)

In [ ]:
# Use papermill to get the target notebook's parameters
def get_params(target):
    params = inspect_notebook(target)
    params = {k: v['default'] for k, v in params.items()}
    return params

# Get parameters from the URL query string in Voila
def get_query_string_params():
    qs = os.getenv('QUERY_STRING', '')
    params = dict(urllib.parse.parse_qsl(qs))
    return params

# Override default params (from notebook parameter cell) with
# query string parameters in Voila.
def merge_params(params, qs_params):
    overridden = []
    for k in params:
        if k in qs_params:
            overridden.append(k)
            params[k] = qs_params[k]
    return overridden
         
# The quick link lets you go back to this page with the user-provided
# parameters as the default values in Voila.
def build_quick_link(target, params):
    myparams = {'target': target}
    for k, v in params.items():
        # Use repr to make sure strings stay quoted on the other side
        myparams[k] = repr(v)
    link = 'notebook-runner.ipynb?' + urllib.parse.urlencode(myparams)
    return f"Quick link: <a href='{link}'>{link}</a>"

# Given a set of parameters, this builds a function that we can give
# to ipywidgets.interactive() and let it build a widget UI for us.
# It returns a widget that we can incorporate into a larger layout.
def build_inputs(target):
    params = get_params(target)
    qs_params = get_query_string_params()
    overridden = merge_params(params, qs_params)
    if overridden:
        with target_input:
            display(HTML(f"<b>Note:</b> Loaded defaults from query string: {', '.join(overridden)}"))
    defaults = ", ".join(f"{k}={v}" for k, v in params.items())
    code = f"def run_notebook({defaults}):\n    parameters = {repr(params)}"
    for p in params:
        code += f"\n    parameters['{p}'] = {p}"
    code += f"\n    quick_link.value = build_quick_link('{target}', parameters)"
    code += f"\n    run_target('{target}', parameters)"
    #print(code)
    exec(code, globals())
    return widgets.interactive(run_notebook, {"manual": True, "manual_name": "Run Notebook"})

In [ ]:
# Load the selected notebook.  This will extract parameters from
# the notebook and display the auto-generated GUI.
def load_notebook(b):
    target = target_name.value
    target_input.clear_output()
    target_output.clear_output()
    w = build_inputs(target)
    with target_input:
        display(w)

In [ ]:
# Widgets for selecting the target notebook
target_select = widgets.Output(layout={'border': '1px solid blue'})
target_name = widgets.Dropdown(options=notebooks, description='Target nb:')
load_button = widgets.Button(description='Load parameters')
load_button.on_click(load_notebook)
with target_select:
    display(target_name)
    display(load_button)
    
# Widgets for auto-generated GUI based on target notebook params
target_input = widgets.Output(layout={'border': '1px solid blue'})
quick_link = widgets.HTML()

# The output from executing the target notebook
target_output = widgets.Output(layout={'border': '1px solid blue'})

# If the target notebook was specified in the URL (Voila only), then go
# ahead and pre-select it and extract the parameters.
qs_params = get_query_string_params()
if 'target' in qs_params:
    target_name.value = qs_params['target']
    with target_select:
        display(HTML(f"<b>Note:</b> Loaded target from query string: {qs_params['target']}"))
    load_notebook(load_button)

display(target_select, target_input, quick_link, target_output)